In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys, kimimaro
import numpy as np
from skimage.morphology import disk, ball, square
import scipy.ndimage as nd
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import cloudvolume as cv
from IPython.display import HTML
from matplotlib.collections import LineCollection

sys.path.append("/home/phornauer/Git/axon_tracking/")
import axon_tracking.skeletonization as skel
import axon_tracking.template_extraction as te
import axon_tracking.visualization as vis

In [ ]:
params = dict()
params['noise_threshold'] = -1
params['abs_threshold'] = -0.2
params['upsample'] = [0.5,0.5,0.5]
params['sampling_rate'] = 20000/params['upsample'][2] # [Hz]
params['ms_cutout'] = [1.5, 5.0]
params['filter_footprint'] = ball(1)#np.ones((2,2,3))
params['max_velocity'] = 5 # [m/s]
params['max_noise_level'] = 0.25
params['el_spacing'] = 17.5 * params['upsample'][2]

In [ ]:
skel_params = dict()
skel_params['scale'] = 3#1
skel_params['const'] = 2#3
skel_params['dust_threshold'] = 5
skel_params['anisotropy'] = (10.0, 10.0, 10.0)
skel_params['tick_threshold'] = 10
skel_params['n_jobs'] = 16

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/concatenated/'
#root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/concatenated/'
stream_id = 'well014'
template_id = 274
template, template_save_file, noise = skel.load_template_file(root_path, stream_id, template_id)
temp_diff = np.diff(template)
capped_template, target_coor= skel.localize_neurons(temp_diff, ms_cutout=params['ms_cutout'])

tmp_filt = nd.gaussian_filter(capped_template,sigma=1)

interp_temp = skel.interpolate_template(tmp_filt,spacing=params['upsample'],overwrite=True)
noise = skel.interpolate_template(noise,spacing=params['upsample'][:2] +[1])
th_template = skel.threshold_template(interp_temp, noise, target_coor, params)

vis.plot_template_and_noise(tmp_filt,noise,th_template)

In [ ]:
t_cap = [0, th_template.shape[2]] #in samples
skels = skel.skeletonize(th_template[:,:,t_cap[0]:t_cap[1]].astype('bool'), **skel_params)

skeleton = kimimaro.join_close_components(skels[1], radius=10)

skeleton = kimimaro.postprocess(skeleton, tick_threshold=5, dust_threshold=10)

skeleton = kimimaro.join_close_components(skeleton, radius=200)

#skeleton = kimimaro.postprocess(skeleton, tick_threshold=5, dust_threshold=100)

skeleton = skeleton.consolidate()
skel_paths = cv.Skeleton.paths(skeleton)

In [ ]:
skel_paths[1].shape

In [ ]:
vis.plot_delay_skeleton(skel_paths, params, skel_params,figsize=4, plot_ais=False, plot_ais_connection=False)

In [ ]:
ani = vis.generate_propagation_gif(interp_temp[:,:,6:], params, cumulative=False, vertices=adj_voxels, downsample=2, clim=[-2, -0.15], cmap="Greys", marker_size=0.1)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
ani.save("/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/concatenated/well014/274_skeleton.gif",
         fps=20,dpi=300)

In [ ]:
def generate_voxels_from_points(points, resolution=100):
    """
    Generates voxels (integer coordinates) through which the edges, formed by consecutive points, pass.
    
    Parameters:
    - points: An array of shape (N, 3) where N is the number of points, with each point having 3D coordinates.
    - resolution: The number of interpolation steps along each edge. Higher resolution
                  increases the chance of capturing all voxels an edge passes through.
    
    Returns:
    - voxels: A numpy array of unique voxels (integer coordinates) through which the edges pass.
    """
    voxel_set = set()
    # Iterate over pairs of consecutive points to form edges
    for i in range(len(points) - 1):
        start, end = points[i], points[i + 1]
        # Ensure start and end are numpy arrays for element-wise operations
        start, end = np.array(start), np.array(end)
        # Linearly interpolate between start and end points
        for t in np.linspace(0, 1, resolution):
            voxel = np.round(start + t * (end - start)).astype(int)  # Round to nearest int and ensure type int
            voxel_set.add(tuple(voxel))
    # Convert the set of tuples back to a numpy array
    voxels = np.array(list(voxel_set))
    return voxels

In [ ]:
all_voxels = np.empty((0,3))
for path in skel_paths:
    voxels = generate_voxels_from_points(path,resolution=10)
    all_voxels = np.concatenate((all_voxels, voxels),axis=0)

In [ ]:
adj_voxels = all_voxels
adj_voxels = adj_voxels - np.array([0,0,6])

In [ ]:
all_voxels[1,:]

In [ ]:
adj_voxels[1,:]